# Setup

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import csv

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import ipywidgets as widgets

from flask import Flask, jsonify


In [2]:
# create and save database
#execfile("Setup and Organization/Database Setup.py")

In [3]:
# create engine to newly created database.sqlite
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()


# Explore Database

In [4]:
# explore database
inspector = inspect(engine)
inspector.get_table_names()

['activity',
 'bmi',
 'gender',
 'health',
 'occupation',
 'person',
 'sleep',
 'sleep_disorder']

In [5]:
inspector.get_columns('activity')

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'person_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'activity_level',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'daily_steps',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

# Reflect and query data

In [6]:
# reflect an existing database into a new model
base= automap_base()
# reflect the tables
base.prepare(autoload_with=engine)

In [7]:
base.classes.keys()

[]

In [8]:
base.metadata.tables

FacadeDict({'activity': Table('activity', MetaData(), Column('index', BIGINT(), table=<activity>), Column('id', BIGINT(), table=<activity>), Column('person_id', BIGINT(), table=<activity>), Column('activity_level', BIGINT(), table=<activity>), Column('daily_steps', BIGINT(), table=<activity>), schema=None), 'bmi': Table('bmi', MetaData(), Column('index', BIGINT(), table=<bmi>), Column('id', BIGINT(), table=<bmi>), Column('bmi_name', TEXT(), table=<bmi>), schema=None), 'gender': Table('gender', MetaData(), Column('index', BIGINT(), table=<gender>), Column('id', BIGINT(), table=<gender>), Column('gender_name', TEXT(), table=<gender>), schema=None), 'health': Table('health', MetaData(), Column('index', BIGINT(), table=<health>), Column('id', BIGINT(), table=<health>), Column('person_id', BIGINT(), table=<health>), Column('blood_pressure', TEXT(), table=<health>), Column('stress_level', BIGINT(), table=<health>), Column('heart_rate', BIGINT(), table=<health>), Column('systolic', BIGINT(), 

In [9]:
for mappedclass in base.classes:
    print(mappedclass)



In [10]:
for mdtable in base.metadata.tables:
    print(mdtable)

activity
bmi
gender
health
occupation
person
sleep
sleep_disorder


In [11]:
health_metrics = pd.read_sql(f'select * from person as p \
                             JOIN health as h on p.id = h.person_id \
                             JOIN bmi on h.bmi_id = bmi.id \
                             JOIN gender as g on p.gender_id = g.id', conn)
health_metrics

,index,id,gender_id,age,occupation_id,index,id,person_id,blood_pressure,stress_level,heart_rate,systolic,diastolic,bmi_id,index,id,bmi_name,index,id,gender_name
0,0,1,0,27,0,0,0,1,126/83,6,77,126,83,0,0,0,Overweight,0,0,Male
1,1,2,0,28,1,1,1,2,125/80,8,75,125,80,1,1,1,Normal,0,0,Male
2,2,3,0,28,1,2,2,3,125/80,8,75,125,80,1,1,1,Normal,0,0,Male
3,3,4,0,28,2,3,3,4,140/90,8,85,140,90,2,2,2,Obese,0,0,Male
4,4,5,0,28,2,4,4,5,140/90,8,85,140,90,2,2,2,Obese,0,0,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,369,370,1,59,4,369,369,370,140/95,3,68,140,95,0,0,0,Overweight,1,1,Female
370,370,371,1,59,4,370,370,371,140/95,3,68,140,95,0,0,0,Overweight,1,1,Female
371,371,372,1,59,4,371,371,372,140/95,3,68,140,95,0,0,0,Overweight,1,1,Female
372,372,373,1,59,4,372,372,373,140/95,3,68,140,95,0,0,0,Overweight,1,1,Female


In [12]:
health_metrics = health_metrics[['person_id','heart_rate','systolic','diastolic','gender_name']]
health_metrics

,person_id,heart_rate,systolic,diastolic,gender_name
0,1,77,126,83,Male
1,2,75,125,80,Male
2,3,75,125,80,Male
3,4,85,140,90,Male
4,5,85,140,90,Male
...,...,...,...,...,...
369,370,68,140,95,Female
370,371,68,140,95,Female
371,372,68,140,95,Female
372,373,68,140,95,Female


In [ ]:
h_diag = sns.PairGrid(data=health_metrics.loc[health_metrics['gender_name']==gender,['person_id','heart_rate','systolic','diastolic']])

In [26]:
genderBox = widgets.Dropdown(
    options = ['Male & Female','Male', 'Female'], #'Male & Female',
    value = 'Male & Female',
    description = 'Gender'
)

def health_plot(gender):
    if (gender == 'Male & Female'): #total population
        mask = (health_metrics['gender_name'].isin(['Male', 'Female']))
    else: # male or female
        mask = (health_metrics['gender_name'] == gender)
    h_diag = sns.PairGrid(data=health_metrics.loc[mask,['person_id','heart_rate','systolic','diastolic']])
    h_diag.map_diag(sns.histplot)
    h_diag.map_offdiag(sns.scatterplot)
    #plt.show()
outh = widgets.interactive_output(health_plot, {'gender': genderBox})
display(genderBox, outh)


Dropdown(description='Gender', options=('Male & Female', 'Male', 'Female'), value='Male & Female')

Output()

In [14]:
sns.__version__

'0.13.2'